In [ ]:
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.llms.openai import OpenAI
import openai
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# index paths
strategy_path = "../index/strategy"
gameinfo_path = "../index/game"
altogether_path = "../index/altogether"

In [ ]:
class RAG_agent:
    def __init__(self, index_path):
        storage_context = StorageContext.from_defaults(persist_dir=index_path)
        index = load_index_from_storage(storage_context)
        llm = OpenAI(
            model="gpt-4o",
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0.2,
            num_outputs=1000,
        )
        self.query_engine = index.as_query_engine(response_mode="refine", similarity_top_k=5, verbose = True, llm = llm)

    def refine_prompt(self, query):
        return self.query_engine.query(query)


In [ ]:
# agent to refine original prompt
prompt_refine_agent = openai.OpenAI(api_key=openai.api_key)

original_query= "Recommended full set of equipment for playing with Unsheathe to beat Margit"

messages = [
    {"role": "system",
    "content": "You are a prompt refining agent that helps to refine the prompt to another agent. \
        The job of the other agent is to recommend strategy based on player's status.\
        The game is Elden Ring. \
        Please use vocabularies from the Elden Ring to refine the prompt when suitable.\
        Given the below prompt, refine it so it is better to be feed into the recommendation agent that recommend a strategy."
    },
    {
    "role": "user", 
    "content": f"{original_query}"
    }
]
response = prompt_refine_agent.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    max_tokens=1000,
    n=1,
    top_p=0.95,
    frequency_penalty=0.2,
    temperature=0.7
)

query_to_recommendation_agent = response.choices[0].message.content
print(query_to_recommendation_agent)

In [ ]:
print(response)